# 92_fig2_transition_commitment_vs_discretion

Figure 2: forced transition normal→bad, commitment vs discretion, deterministic innovations (eps=0).

In [ ]:
import os, sys, json, numpy as np, torch, matplotlib.pyplot as plt
sys.path.append("..")

from src.config import ModelParams
from src.io_utils import load_json, load_npz, load_torch, load_selected_run, find_latest_run_dir
from src.deqn import PolicyNetwork

ART = os.path.join("..","artifacts","runs")

def get_run(policy: str) -> str:
    rd = load_selected_run(ART, policy)
    if rd is None:
        rd = find_latest_run_dir(ART, policy)
    if rd is None:
        raise RuntimeError(f"No runs found for policy={policy} under {ART}")
    return rd

def _parse_dtype(s: str):
    if s is None:
        return torch.float32
    if isinstance(s, torch.dtype):
        return s
    s = str(s)
    if "float64" in s:
        return torch.float64
    if "float32" in s:
        return torch.float32
    if "bfloat16" in s:
        return torch.bfloat16
    return torch.float32

def load_params_from_run(run_dir: str, *, device="cpu"):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    p = cfg.get("params", {})
    dtype = _parse_dtype(p.get("dtype"))
    dev = device if device is not None else p.get("device","cpu")
    keep = {k:v for k,v in p.items() if k in ModelParams.__dataclass_fields__}
    keep["device"] = dev
    keep["dtype"] = dtype
    return ModelParams(**keep).to_torch()

def load_net_from_run(run_dir: str, d_in: int, d_out: int):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    tc = cfg.get("train_cfg", {})
    hidden = tuple(tc.get("hidden_layers", (512,512)))
    activation = tc.get("activation", "selu")
    net = PolicyNetwork(d_in, d_out, hidden=hidden, activation=activation)
    state = load_torch(os.path.join(run_dir, "weights.pt"), map_location="cpu")
    # state is usually a plain state_dict
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    net.load_state_dict(state)
    net.eval()
    return net

# --- paper reporting helpers ---
ann = lambda x: 400.0*x  # annualized percent (quarterly -> annual)


In [ ]:
from src.steady_states import solve_commitment_sss_from_policy, solve_discretion_sss_from_policy
from src.experiments import DeterministicPathSpec, simulate_deterministic_path

run_comm = get_run("commitment")
run_disc = get_run("discretion")

params_comm = load_params_from_run(run_comm)
params_disc = load_params_from_run(run_disc)

# In the paper, policies are compared under the same calibration.
def _assert_same_calibration(p1, p2, *, atol=1e-12):
    fields = [k for k in ModelParams.__dataclass_fields__.keys() if k not in ("device", "dtype")]
    diffs = []
    for k in fields:
        v1 = getattr(p1, k)
        v2 = getattr(p2, k)
        if isinstance(v1, (int, float)) and isinstance(v2, (int, float)):
            if abs(float(v1) - float(v2)) > atol:
                diffs.append((k, v1, v2))
        else:
            if v1 != v2:
                diffs.append((k, v1, v2))
    if diffs:
        details = ", ".join([f"{k}: commitment={a}, discretion={b}" for k, a, b in diffs])
        raise RuntimeError(
            "Figure 2 requires identical calibration across commitment/discretion runs. "
            f"Differences found: {details}"
        )

_assert_same_calibration(params_comm, params_disc)
params = params_comm

net_comm = load_net_from_run(run_comm, 7, 13)
net_disc = load_net_from_run(run_disc, 5, 11)

comm_sss = solve_commitment_sss_from_policy(params, net_comm)
disc_sss = solve_discretion_sss_from_policy(params, net_disc)

x0_comm = torch.tensor([[float(comm_sss.by_regime[0]["Delta_prev"]), float(comm_sss.by_regime[0]["logA"]), float(comm_sss.by_regime[0]["loggtilde"]), float(comm_sss.by_regime[0]["xi"]), 0.0,
                         float(comm_sss.by_regime[0]["vartheta_prev"]),
                         float(comm_sss.by_regime[0]["varrho_prev"])]], dtype=torch.float32)
x0_disc = torch.tensor([[float(disc_sss.by_regime[0]["Delta_prev"]), float(disc_sss.by_regime[0]["logA"]), float(disc_sss.by_regime[0]["loggtilde"]), float(disc_sss.by_regime[0]["xi"]), 0.0]], dtype=torch.float32)

T=40
spec = DeterministicPathSpec(T=T, epsA=0.0, epsg=0.0, epst=0.0, regime_path=[0]+[1]*T)

path_comm = simulate_deterministic_path(params, "commitment", net_comm, x0=x0_comm, spec=spec, compute_implied_i=True)
path_disc = simulate_deterministic_path(params, "discretion", net_disc, x0=x0_disc, spec=spec, compute_implied_i=True)

# regime_path=[0]+[1]*T sets s_{t+1}; switch impact is at index 1.
t = np.arange(T)
pi_comm = path_comm["pi"][1:,0]
pi_disc = path_disc["pi"][1:,0]

plt.figure()
plt.plot(t, ann(pi_comm), label="Commitment")
plt.plot(t, ann(pi_disc), label="Discretion", linestyle="--")
plt.axhline(0, linewidth=1)
plt.title("Figure 2: Inflation transition normal->bad")
plt.xlabel("t"); plt.ylabel("pi"); plt.legend(); plt.show()
